In [2]:

# utility file for reading distance matrices stored in basic PHYLIP format (http://www.phylo.org/tools/obsolete/phylip.html)

def parse(distance_matrix):
    """Read matrix of distances form Phylip-formatted file.(http://www.phylo.org/tools/obsolete/phylip.html)

    Return list of names and dictionary structure such that
    d[a][b] is the distance between names[a] and names[b].
    """
    try:
        #with open(filename) as src:
        #    lines = [line.strip() for line in src if line.strip()]
        lines = distance_matrix.split("\n")
        try:
            n = int(lines[0])
        except ValueError:
            raise ValueError('First line of file should contain number of samples')

        if n < 2:
            raise ValueError('N must be 2 or more')

        if len(lines) < 1+n:
            raise ValueError('Not enough lines of data')

        names = [line.split()[0] for line in lines[1:1+n]]
        dist = []
        for k in range(n):
            pieces = lines[1+k].split()[1:]
            try:
                vals = [float(v) for v in pieces]
            except ValueError:
                raise ValueError('invalid distance on line ' + str(2+k))

            if len(vals) != n:
                raise ValueError('line %d should have %d entries' % (2+k,n))

            dist.append( { j : vals[j] for j in range(n) } )
    except IOError:
        print('Unable to open file:',filename)
        raise

    return names,dist



In [3]:
####### Example for loading distance matrix using basic PHYLIP format (http://www.phylo.org/tools/obsolete/phylip.html)

# Line number 1 provides number of species
# Rest rows are distance matrix
# The parse function will return a list containing the species names, and a disctionary containing distance matrix

dist = """5
A 0	20	60	100	90
B 20	0	50	90	80
C 60	50	0	40	50
D 100	90	40	0	30
E 90	80	50	30	0"""

names,dist = parse(dist)
N = len(names)               # number of leaves

print("Number of leaves: ", N)
print("The leaf nodes: ", names)
print("Distance matrix: ", dist)


Number of leaves:  5
The leaf nodes:  ['A', 'B', 'C', 'D', 'E']
Distance matrix:  [{0: 0.0, 1: 20.0, 2: 60.0, 3: 100.0, 4: 90.0}, {0: 20.0, 1: 0.0, 2: 50.0, 3: 90.0, 4: 80.0}, {0: 60.0, 1: 50.0, 2: 0.0, 3: 40.0, 4: 50.0}, {0: 100.0, 1: 90.0, 2: 40.0, 3: 0.0, 4: 30.0}, {0: 90.0, 1: 80.0, 2: 50.0, 3: 30.0, 4: 0.0}]


In [4]:
####### Block 4: Do not change anything in this block  #######
def clustering_main(names, dist, compute_distance):
    #--------------------------------------------------------
    # Initialize Data Structures to save clusters, trees, and nodes
    #--------------------------------------------------------

    # clusters[k] is a list of original leaf IDs for those
    # leaves within cluster k, i.e. [[1], [2], [3]]
    clusters = []
    for k in range(N):
        clusters.append( [k] )   # cluster k originally has only leaf k


    # tree[k] is our tuple representation of cluster k, i.e., ('A',(),())
    trees = []
    for k in range(N):
        trees.append( (names[k], (), ()) )



    # originally, clusters 0 through N-1 are the active ones to merge and calculate distance
    activeClusters = set( range(N) )


    #--------------------------------------------------------
    # Main clustering algorithm
    #--------------------------------------------------------

    ##### Keep merging clusters until one cluster left
    while len(activeClusters) > 1:

        #### step 1: find the 'closest' pair from distance matrix and return index. 
        #### Need define function   find_nearest_active()

        a,b = find_nearest_active(dist, activeClusters)


        #### step 2: remove a and b from active set, a and b is the index of node
        activeClusters.remove(a)
        activeClusters.remove(b)

        #### reporting merging information
        print('About to merge clusters at distance %.3f:' % dist[a][b])
        print('  First:  ' + str(trees[a]))
        print('  Second: ' + str(trees[b]))


        # new cluster has all leaves of clusters[a] and clusters[b]
        clusters.append(clusters[a] + clusters[b])  # combination of both subtree, i.e.  [1] + [2] = [1,2]

        #### step 4: create new tree representation for this cluster. 
        #### Need define function     combine_trees()
        trees.append( combine_trees(dist, trees,a,b) )

        print('  Result: ', trees[-1])

        #### step 5: compute distances between all other clusters to the new clade
        newID = len(clusters)-1

        dist.append( {} )    # dictionary for new cluster's distances to other clusters
        for c in activeClusters:
            val = compute_distance(dist, clusters,c, newID) #### Need define compute_distance function
            dist[newID][c] = val
            dist[c][newID] = val

        print('  Distances to new cluster:')
        for c in activeClusters:
            print('    %.2f: %s' % (dist[newID][c], trees[c]))

        #### step 6: officially add new cluster to active set, and repeat the procedure
        activeClusters.add(newID)
    
    final_tree = trees[-1]
    return final_tree

In [17]:
#--------------------------------------------------------
# Utility functions (to be completed by student)
#--------------------------------------------------------


#--------------------------------------------------------
# Summary of data structures (created later in this script).
# All indexing is ZERO-based (i.e. names[0] is the first leaf)
#
# names[k] is the original name of the k-th leaf of the input
#
# clusters[k] is a list of integers that ID the leaves of the cluster
#
# trees[k] is a tuple representation of the subtree for clades[k]
#
# activeClusters is a list of integer IDs for active clusters during the
# clustering process
#
# dist is list of dictionaries such that dist[a][b] is distance
# between clusters a and b
#
#--------------------------------------------------------


def combine_trees(dist,trees, a,b):
    """Return tuple that represents tree for newly merged clusters a and b.
    For instance, a is ('A',(),()), b is ('B',()),()), and the distance between A and B in dist matrix is 0.5,
    The combined tree should be (0.5,('A',(),()), ('B',(),()))
    
    You need write the correct codes for this function.
    """
    return (dist[a][b]/2,trees[a],trees[b])

def find_nearest_active(dist,activeClusters):
    """Return IDs (index) for the two nearest active clusters in the matrix. 
    For instance, if i-th row and j-th column give the smallest number, just return i,j"""
    min_value = float('inf')
    a = 0
    b = 0
    for cluster1 in activeClusters:
      for cluster2 in activeClusters:
        if cluster1<cluster2:
          cluster_distance = dist[cluster1][cluster2]
          if cluster_distance < min_value:
            min_value = cluster_distance
            a  = cluster1
            b  = cluster2 
    return a,b

def single_linkage_cost(dist, clusters, a,b):
    """Return distance between clusters a and b.

    For single linkage, distance should be based on the
    NEAREST neighbors across the clusters.
    """
    clusterA = clusters[a]
    clusterB = clusters[b]

    min_value = float('inf')
    for node1 in clusterA:
      for node2 in clusterB:
         if dist[node1][node2] < min_value:
           min_value = dist[node1][node2]
    
    return min_value




def complete_linkage_cost(dist, clusters, a, b):
    """Return distance between clusters a and b.

    For complete linkage, distance should be based on the
    FURTHEST neighbors across the clusters.
    """
    clusterA = clusters[a]
    clusterB = clusters[b]

    max_value = float('-inf')
    for node1 in clusterA:
      for node2 in clusterB:
         if dist[node1][node2] > max_value:
           max_value = dist[node1][node2]
    
    return max_value



def upgma_linkage_cost(dist, clusters, a, b):
    """Return distance between clusters a and b.

    For UPGMA, distance should be the unweighted average of
    distances for all pairs across the clusters.
    """
    clusterA = clusters[a]
    clusterB = clusters[b]

    avg_value = 0
    total_num = 0
    for node1 in clusterA:
      for node2 in clusterB:
         avg_value = avg_value + dist[node1][node2]
         total_num = total_num + 1
    
    avg_value = avg_value/total_num
    
    return avg_value

In [18]:
####### example: Load distance matrix

dist = """5
A 0	20	60	100	90
B 20	0	50	90	80
C 60	50	0	40	50
D 100	90	40	0	30
E 90	80	50	30	0"""

names,dist = parse(dist)
N = len(names)               # number of leaves

print("Number of leaves: ", N)
print("The leaf nodes: ", names)
print("Distance matrix: ", dist)


Number of leaves:  5
The leaf nodes:  ['A', 'B', 'C', 'D', 'E']
Distance matrix:  [{0: 0.0, 1: 20.0, 2: 60.0, 3: 100.0, 4: 90.0}, {0: 20.0, 1: 0.0, 2: 50.0, 3: 90.0, 4: 80.0}, {0: 60.0, 1: 50.0, 2: 0.0, 3: 40.0, 4: 50.0}, {0: 100.0, 1: 90.0, 2: 40.0, 3: 0.0, 4: 30.0}, {0: 90.0, 1: 80.0, 2: 50.0, 3: 30.0, 4: 0.0}]


In [19]:
###### Get tree using single linkage


trees_single = clustering_main(names, dist, single_linkage_cost)

print()
print('Final tree using single linkage:',trees_single)
print()

About to merge clusters at distance 20.000:
  First:  ('A', (), ())
  Second: ('B', (), ())
  Result:  (10.0, ('A', (), ()), ('B', (), ()))
  Distances to new cluster:
    50.00: ('C', (), ())
    90.00: ('D', (), ())
    80.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('D', (), ())
  Second: ('E', (), ())
  Result:  (15.0, ('D', (), ()), ('E', (), ()))
  Distances to new cluster:
    40.00: ('C', (), ())
    80.00: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 40.000:
  First:  ('C', (), ())
  Second: (15.0, ('D', (), ()), ('E', (), ()))
  Result:  (20.0, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Distances to new cluster:
    50.00: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 50.000:
  First:  (10.0, ('A', (), ()), ('B', (), ()))
  Second: (20.0, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Result:  (25.0, (10.0, ('A', (), ()), ('B', (), ())), (20.0, ('C', (), ()), (15.0, ('D', ()

In [20]:
###### Get tree using complete linkage

trees_complete = clustering_main(names, dist, complete_linkage_cost)
trees_UPGMA = clustering_main(names, dist, upgma_linkage_cost)

print()
print('Final tree using complete linkage:',trees_complete)
print()

About to merge clusters at distance 20.000:
  First:  ('A', (), ())
  Second: ('B', (), ())
  Result:  (10.0, ('A', (), ()), ('B', (), ()))
  Distances to new cluster:
    60.00: ('C', (), ())
    100.00: ('D', (), ())
    90.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('D', (), ())
  Second: ('E', (), ())
  Result:  (15.0, ('D', (), ()), ('E', (), ()))
  Distances to new cluster:
    50.00: ('C', (), ())
    100.00: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 50.000:
  First:  ('C', (), ())
  Second: (15.0, ('D', (), ()), ('E', (), ()))
  Result:  (25.0, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Distances to new cluster:
    100.00: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 100.000:
  First:  (10.0, ('A', (), ()), ('B', (), ()))
  Second: (25.0, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Result:  (50.0, (10.0, ('A', (), ()), ('B', (), ())), (25.0, ('C', (), ()), (15.0, ('D'

In [21]:
###### Block 8: Get tree using upgma linkage

trees_UPGMA = clustering_main(names, dist, upgma_linkage_cost)

print()
print('Final tree using upgma linkage:',trees_UPGMA)
print()


About to merge clusters at distance 20.000:
  First:  ('A', (), ())
  Second: ('B', (), ())
  Result:  (10.0, ('A', (), ()), ('B', (), ()))
  Distances to new cluster:
    55.00: ('C', (), ())
    95.00: ('D', (), ())
    85.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('D', (), ())
  Second: ('E', (), ())
  Result:  (15.0, ('D', (), ()), ('E', (), ()))
  Distances to new cluster:
    45.00: ('C', (), ())
    90.00: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 45.000:
  First:  ('C', (), ())
  Second: (15.0, ('D', (), ()), ('E', (), ()))
  Result:  (22.5, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Distances to new cluster:
    78.33: (10.0, ('A', (), ()), ('B', (), ()))
About to merge clusters at distance 78.333:
  First:  (10.0, ('A', (), ()), ('B', (), ()))
  Second: (22.5, ('C', (), ()), (15.0, ('D', (), ()), ('E', (), ())))
  Result:  (39.166666666666664, (10.0, ('A', (), ()), ('B', (), ())), (22.5, ('C', (), ()), (

In this lab I was able to understand the code and the purpose of each of the defined alogrithms. However, I struggled with defining the functions. Specifically, I struggled with creating a for loop to find the minimum value in the matrix. The variables I struggled to define were the finding_active_clusters and the single_linkage_cost functions. Once I observed the process for the finding the minimum values, I was able to comperehensively understand and predict the code to determine the maximum and average values for the code. 